In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn sklearn matplotlib

# Creating a model to detect different hand gestures action. 
# We have 8 hand gestures. We test with the right hand only.
# Currently we have 11 testers.

import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
def draw_styled_landmarks(image, results):
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([rh])
#-----------------------------------------------------------------------------------------------------------------
# Create Folders for data collection

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data')

# Actions that we try to detect
actions = np.array(['Move Up', 'Move Down', 'Move Right', 'Move Left', 'Rotate Right', 'Rotate Left', 'Zoom In', 'Zoom Out'])

# 11 Testers
testers = ['Tester1', 'Tester2', 'Tester3', 'Tester4', 'Tester5', 'Tester6', 'Tester7', 'Tester8', 'Tester9', 'Tester10', 'Tester11']

# Define a dictionary to store the number of sequences for each tester
no_sequences = {
    'Tester1': 50,
    'Tester2': 50,
    'Tester3': 20,
    'Tester4': 20,
    'Tester5': 20,
    'Tester6': 20,
    'Tester7': 20,
    'Tester8': 20,
    'Tester9': 20,
    'Tester10': 20,
    'Tester11': 20
}

# Videos are going to be 30 frames in length
sequence_length = 30

#--------------------------------------------------------------------------------------------
# labelling

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []

# Combine the lists of sequences and labels
sequences, labels = [], []
for action_idx, action in enumerate(actions):
    for tester in testers:
        num_sequences = no_sequences[tester]
        for sequence in range(num_sequences):
            sequence_path = os.path.join(DATA_PATH, action, tester, str(sequence))
            if os.path.exists(sequence_path):
                window = []
                for frame_num in range(sequence_length):
                    file_path = os.path.join(sequence_path, "{}.npy".format(frame_num))
                    if os.path.exists(file_path):
                        res = np.load(file_path)
                        window.append(res)
                if len(window) == sequence_length:
                    sequences.append(window)
                    labels.append(label_map[action])

X = np.array(sequences)        
y = to_categorical(labels).astype(int)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Model

# LSTM

In [3]:
X.shape

(2239, 30, 63)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
56/56 [==============================] - 7s 52ms/step - loss: 2.0417 - categorical_accuracy: 0.1932
Epoch 2/2000
56/56 [==============================] - 2s 41ms/step - loss: 2.0221 - categorical_accuracy: 0.2071
Epoch 3/2000
56/56 [==============================] - 2s 42ms/step - loss: 1.8802 - categorical_accuracy: 0.2468
Epoch 4/2000
56/56 [==============================] - 2s 43ms/step - loss: 1.6789 - categorical_accuracy: 0.3473
Epoch 5/2000
56/56 [==============================] - 2s 39ms/step - loss: 1.5165 - categorical_accuracy: 0.4149
Epoch 6/2000
56/56 [==============================] - 3s 45ms/step - loss: 1.3576 - categorical_accuracy: 0.4657
Epoch 7/2000
56/56 [==============================] - 3s 51ms/step - loss: 1.2375 - categorical_accuracy: 0.5126
Epoch 8/2000
56/56 [==============================] - 3s 54ms/step - loss: 1.3169 - categorical_accuracy: 0.4880
Epoch 9/2000
56/56 [==============================] - 3s 47ms/step - loss: 1.2118 - categorical_

KeyboardInterrupt: 

In [6]:
model.summary()
print("****************************************************")
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
model.save('lstm_model.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            32768     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 8)                 264       
                                                                 
Total params: 187,496
Trainable params: 187,496
Non-tr

# LSTM model 2

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback, early_stop], validation_data=(X_test, y_test))

model.summary()
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
model.save('lstm_model2.h5')

Epoch 1/2000
56/56 [==============================] - 7s 54ms/step - loss: 2.0229 - categorical_accuracy: 0.1474 - val_loss: 1.9580 - val_categorical_accuracy: 0.2612
Epoch 2/2000
56/56 [==============================] - 2s 44ms/step - loss: 1.9091 - categorical_accuracy: 0.2948 - val_loss: 1.8979 - val_categorical_accuracy: 0.2656
Epoch 3/2000
56/56 [==============================] - 2s 44ms/step - loss: 1.8123 - categorical_accuracy: 0.3439 - val_loss: 1.7715 - val_categorical_accuracy: 0.3415
Epoch 4/2000
56/56 [==============================] - 3s 45ms/step - loss: 1.6794 - categorical_accuracy: 0.4098 - val_loss: 1.6060 - val_categorical_accuracy: 0.3705
Epoch 5/2000
56/56 [==============================] - 3s 45ms/step - loss: 1.5142 - categorical_accuracy: 0.4394 - val_loss: 1.5347 - val_categorical_accuracy: 0.4196
Epoch 6/2000
56/56 [==============================] - 3s 47ms/step - loss: 1.3960 - categorical_accuracy: 0.4791 - val_loss: 1.3909 - val_categorical_accuracy: 0.482

Epoch 50/2000
56/56 [==============================] - 3s 56ms/step - loss: 0.3534 - categorical_accuracy: 0.8649 - val_loss: 0.6557 - val_categorical_accuracy: 0.8080
Epoch 51/2000
56/56 [==============================] - 3s 61ms/step - loss: 0.3162 - categorical_accuracy: 0.8660 - val_loss: 0.6895 - val_categorical_accuracy: 0.8125
Epoch 52/2000
56/56 [==============================] - 3s 55ms/step - loss: 0.4266 - categorical_accuracy: 0.8420 - val_loss: 0.6531 - val_categorical_accuracy: 0.7857
Epoch 53/2000
56/56 [==============================] - 3s 58ms/step - loss: 0.3440 - categorical_accuracy: 0.8716 - val_loss: 0.5881 - val_categorical_accuracy: 0.8281
Epoch 54/2000
56/56 [==============================] - 3s 60ms/step - loss: 0.3206 - categorical_accuracy: 0.8788 - val_loss: 0.6351 - val_categorical_accuracy: 0.8125
Epoch 55/2000
56/56 [==============================] - 3s 59ms/step - loss: 0.2877 - categorical_accuracy: 0.8889 - val_loss: 0.7655 - val_categorical_accuracy:

# CNN

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import numpy as np

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], X.shape[3])))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback, early_stop], validation_data=(X_test, y_test))
model.summary()
print("****************************************************")
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
model.save('cnn_model_test.h5')

Epoch 1/2000
56/56 [==============================] - 3s 46ms/step - loss: 2.0612 - accuracy: 0.1887 - val_loss: 2.0043 - val_accuracy: 0.2701
Epoch 2/2000
56/56 [==============================] - 2s 39ms/step - loss: 1.9038 - accuracy: 0.2669 - val_loss: 1.7837 - val_accuracy: 0.3862
Epoch 3/2000
56/56 [==============================] - 2s 41ms/step - loss: 1.7487 - accuracy: 0.3540 - val_loss: 1.6536 - val_accuracy: 0.4018
Epoch 4/2000
56/56 [==============================] - 2s 44ms/step - loss: 1.6093 - accuracy: 0.4204 - val_loss: 1.4803 - val_accuracy: 0.5067
Epoch 5/2000
56/56 [==============================] - 2s 40ms/step - loss: 1.4726 - accuracy: 0.4746 - val_loss: 1.3826 - val_accuracy: 0.5134
Epoch 6/2000
56/56 [==============================] - 2s 40ms/step - loss: 1.3167 - accuracy: 0.5366 - val_loss: 1.2581 - val_accuracy: 0.5446
Epoch 7/2000
56/56 [==============================] - 2s 40ms/step - loss: 1.2547 - accuracy: 0.5304 - val_loss: 1.2128 - val_accuracy: 0.5580

Epoch 58/2000
56/56 [==============================] - 2s 43ms/step - loss: 0.2722 - accuracy: 0.9112 - val_loss: 0.5536 - val_accuracy: 0.8304
Epoch 59/2000
56/56 [==============================] - 2s 42ms/step - loss: 0.2871 - accuracy: 0.9012 - val_loss: 0.5352 - val_accuracy: 0.8460
Epoch 60/2000
56/56 [==============================] - 2s 43ms/step - loss: 0.2779 - accuracy: 0.9068 - val_loss: 0.5258 - val_accuracy: 0.8638
Epoch 61/2000
56/56 [==============================] - 3s 45ms/step - loss: 0.2576 - accuracy: 0.9107 - val_loss: 0.4868 - val_accuracy: 0.8482
Epoch 62/2000
56/56 [==============================] - 2s 44ms/step - loss: 0.2493 - accuracy: 0.9135 - val_loss: 0.5301 - val_accuracy: 0.8438
Epoch 63/2000
56/56 [==============================] - 2s 42ms/step - loss: 0.2455 - accuracy: 0.9168 - val_loss: 0.5208 - val_accuracy: 0.8438
Epoch 64/2000
56/56 [==============================] - 2s 43ms/step - loss: 0.2383 - accuracy: 0.9185 - val_loss: 0.4742 - val_accuracy:

# CNN Model (2)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
import numpy as np

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], X.shape[3])))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback, early_stop], validation_data=(X_test, y_test))
model.summary()
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
model.save('cnn_model2.h5')

Epoch 1/2000
56/56 [==============================] - 3s 42ms/step - loss: 2.0141 - accuracy: 0.2111 - val_loss: 1.9305 - val_accuracy: 0.2232
Epoch 2/2000
56/56 [==============================] - 2s 37ms/step - loss: 1.8103 - accuracy: 0.3311 - val_loss: 1.7080 - val_accuracy: 0.4129
Epoch 3/2000
56/56 [==============================] - 2s 37ms/step - loss: 1.5962 - accuracy: 0.4305 - val_loss: 1.5412 - val_accuracy: 0.4598
Epoch 4/2000
56/56 [==============================] - 2s 37ms/step - loss: 1.4192 - accuracy: 0.4975 - val_loss: 1.3378 - val_accuracy: 0.5536
Epoch 5/2000
56/56 [==============================] - 2s 39ms/step - loss: 1.2461 - accuracy: 0.5684 - val_loss: 1.2313 - val_accuracy: 0.5938
Epoch 6/2000
56/56 [==============================] - 2s 38ms/step - loss: 1.1525 - accuracy: 0.5991 - val_loss: 1.0865 - val_accuracy: 0.6607
Epoch 7/2000
56/56 [==============================] - 2s 40ms/step - loss: 1.0401 - accuracy: 0.6561 - val_loss: 1.0361 - val_accuracy: 0.6696

Epoch 58/2000
56/56 [==============================] - 2s 39ms/step - loss: 0.2499 - accuracy: 0.9129 - val_loss: 0.4941 - val_accuracy: 0.8460
Epoch 59/2000
56/56 [==============================] - 2s 39ms/step - loss: 0.2648 - accuracy: 0.9107 - val_loss: 0.4683 - val_accuracy: 0.8304
Epoch 60/2000
56/56 [==============================] - 2s 40ms/step - loss: 0.2596 - accuracy: 0.9129 - val_loss: 0.4574 - val_accuracy: 0.8527
Epoch 61/2000
56/56 [==============================] - 2s 39ms/step - loss: 0.2564 - accuracy: 0.9146 - val_loss: 0.4071 - val_accuracy: 0.8661
Epoch 62/2000
56/56 [==============================] - 2s 39ms/step - loss: 0.2485 - accuracy: 0.9140 - val_loss: 0.4062 - val_accuracy: 0.8705
Epoch 63/2000
56/56 [==============================] - 2s 38ms/step - loss: 0.2373 - accuracy: 0.9157 - val_loss: 0.4596 - val_accuracy: 0.8594
Epoch 64/2000
56/56 [==============================] - 2s 38ms/step - loss: 0.2212 - accuracy: 0.9257 - val_loss: 0.4382 - val_accuracy:

# SVM

In [3]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss

# Compute mean and std of landmark coordinates across all frames
X_train_processed = np.array([np.concatenate([np.mean(seq, axis=0), np.std(seq, axis=0)]) for seq in X_train])
X_test_processed = np.array([np.concatenate([np.mean(seq, axis=0), np.std(seq, axis=0)]) for seq in X_test])
y_test = np.argmax(y_test, axis=1) # Convert multilabel-indicator targets to multiclass targets
y_train = np.argmax(y_train)

# Create SVM model and train on training data
model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=10, gamma='scale', probability=True))
model.fit(X_train_processed, y_train)

# Evaluate model on test data
y_pred = model.predict(X_test_processed)
accuracy = accuracy_score(y_test, y_pred)
loss = log_loss(y_test, model.predict_proba(X_test_processed))
print("Test accuracy: ",accuracy)
print('Test loss:', loss)
import pickle
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# with open('svm_model.pkl', 'rb') as file:
#     model = pickle.load(file)

ValueError: y should be a 1d array, got an array of shape () instead.

In [3]:
y_train = np.argmax(y_train, axis=1)

In [4]:
y_train.shape

(1791,)

In [4]:
X.shape

(1200, 30, 258)

In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

X = X_train
# Convert one-hot encoded y_train to 1D array of integers
y = np.argmax(y_train, axis=1)


# flatten the input data
X_flat = X.reshape((X.shape[0], -1))

# define the parameter grid
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}

# create the GridSearchCV object
grid_search = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=5, n_jobs=-1)

# fit the GridSearchCV object to the data
grid_search.fit(X_flat, y)

# print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'C': 10, 'kernel': 'linear'}
Best score: 0.8308211823656648


# Decision Trees

In [ ]:
model.summary()
print("****************************************************")
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)
model.save('descisionTrees_model.h5')

# Testing In Real Time

In [3]:
from scipy import stats

In [6]:
colors = [(245,117,16), (117,245,16), (16,0,245),(255,0,0),(160,117,245),(16,170,245),(250,117,245),(16,117,0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num] , -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

model.load_weights('cnn_model.h5')

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
        word = ""
        if np.argmax(res) < len(actions):
                word = actions[np.argmax(res)]
#             else:
#                 word = ""

#             # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, word, (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1 , (255, 255, 255), 2 , cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 24ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 20ms/step
Move Down
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/s

Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>

1/1 [==============================] - 0s 24ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 21ms/step
Rotate Left
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [===========================

1/1 [==============================] - 0s 24ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Move Up
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
Move Right
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Move Right
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 24ms/step
Move Right
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Move Right
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 23ms/step
Move Left
